In [12]:
import numpy as np

In [13]:
def rotate_state(state: list[str]) -> list[str]:
    rotated_state: list[str] = []

    for i in range(len(state)):
        tmp = ""
        for j in range(len(state)):
            tmp += state[j][i]

        rotated_state.append(tmp)
    rotated_state.reverse()
    return rotated_state


def beatify_state(state: list[str], delim = "<br>") -> str:
    beautified_state: str = ""
    for i in range(len(state)):
        for j in range(len(state)):
            beautified_state += f"{state[i][j]} "
        beautified_state += delim
    return beautified_state


def rotate_and_beatify(state: list[str], delim: str = "<br>") -> str:
    return beatify_state(rotate_state(state), delim)

In [14]:
from enum import Enum


class GoValidity(Enum):
    VALID = "valid"
    GAME_OVER = "gameOver"
    NOT_YOUR_TURN = "notYourTurn"
    POINT_BROKEN = "pointBroken"
    POINT_NOT_EMPTY = "pointNotEmpty"
    NO_SUICIDE = "noSuicide"
    BOARD_REPEATED = "boardRepeated"

In [15]:
import numpy as np
from collections import deque


def rotate_state(state: list[str]) -> list[str]:
    rotated_state: list[str] = []

    for i in range(len(state)):
        tmp = ""
        for j in range(len(state)):
            tmp += state[j][i]

        rotated_state.append(tmp)
    rotated_state.reverse()
    return rotated_state


def beatify_state(state: list[str], delim="<br>") -> str:
    beautified_state: str = ""
    for i in range(len(state)):
        for j in range(len(state)):
            beautified_state += f"{state[i][j]} "
        beautified_state += delim
    return beautified_state


def rotate_and_beatify(state: list[str], delim: str = "<br>") -> str:
    return beatify_state(rotate_state(state), delim)


class Go:
    def __init__(self, board_width: int, board_height: int, board: list[str]):
        if len(board) != 5:
            raise ValueError("board size must be 5x5")
        self.board_width = board_width
        self.board_height = board_height
        self.board_size = self.board_width * self.board_height
        self.history: list[str] = []

        self.str_board: list[str] = board
        self.board: np.ndarray = self.transform_board(board)

    def decode_action(self, action_idx: int):
        board_size = self.board_width * self.board_height
        if action_idx == board_size:
            return (-1, -1)
        else:
            x = action_idx // self.board_height
            y = action_idx % self.board_height
            return (x, y)

    def encode_action(self, x: int, y: int) -> int:
        return x * self.board_height + y

    def transform_board(self, board: list[str]):
        """
        Converts a list of strings (like [".....", "..X..", ...]) into a numpy array
        with the following encoding:
          '.' -> 0 (empty)
          'X' -> 1 (black)
          'O' -> 2 (white)
          '#' -> 3 (blocked or invalid)
        """
        transformed = np.zeros([5, 5], dtype=int)
        for i, row_str in enumerate(board):
            for j, char in enumerate(row_str):
                if char == ".":
                    transformed[i][j] = 0
                elif char == "X":
                    transformed[i][j] = 1
                elif char == "O":
                    transformed[i][j] = 2
                elif char == "#":
                    transformed[i][j] = 3
        return transformed

    def decode_board(self, board_array: np.ndarray):
        """
        Converts a numpy board array (with 0/1/2/3) back into the
        string-based representation for printing or logging.
        """
        decoded_board = []
        for i in range(board_array.shape[0]):
            tmp = ""
            for j in range(board_array.shape[1]):
                val = board_array[i][j]
                if val == 0:
                    tmp += "."
                elif val == 1:
                    tmp += "X"
                elif val == 2:
                    tmp += "O"
                elif val == 3:
                    tmp += "#"
            decoded_board.append(tmp)
        return decoded_board

    def __str__(self):
        board = self.decode_board(self.board)
        return rotate_and_beatify(
            board, "\n"
        )  # Assuming you have a rotate_and_beatify function

    def state_after_action(self, action: int, is_white: bool) -> tuple[np.ndarray, int]:
        """Returns (new_board_state_as_strings, captures)."""
        if action == self.board_size:  # Pass move
            return self.board.copy(), 0

        x, y = self.decode_action(action)
        new_board = self.board.copy()
        color = 2 if is_white else 1
        captures = 0

        # Place the stone
        new_board[x][y] = color

        # Check for captures (opponent stones)
        visited: set[tuple[int, int]] = set()
        opponent_color = 1 if is_white else 2

        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            nx, ny = x + dx, y + dy
            if (
                0 <= nx < self.board_width
                and 0 <= ny < self.board_height
                and new_board[nx][ny] == opponent_color
            ):
                if (nx, ny) not in visited:
                    group = self.find_group(nx, ny, opponent_color, visited, new_board)
                    if not self.has_liberties(group, new_board):
                        # Remove captured group
                        for gx, gy in group:
                            new_board[gx][gy] = 0
                            captures += 1

        return self.decode_board(new_board), captures

    def find_group(
        self, x: int, y: int, color: int, visited: set, board: np.ndarray
    ) -> set[tuple[int, int]]:
        """Find all connected stones of the given color using DFS."""
        if (x, y) in visited:
            return set()
        if not (0 <= x < self.board_width and 0 <= y < self.board_height):
            return set()
        if board[x][y] != color:
            return set()

        visited.add((x, y))
        group = {(x, y)}

        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            group.update(self.find_group(x + dx, y + dy, color, visited, board))

        return group

    def has_liberties(self, group: set[tuple[int, int]], board: np.ndarray) -> bool:
        """Check if any position in the group has an empty adjacent point (a liberty)."""
        for x, y in group:
            for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                nx, ny = x + dx, y + dy
                if (
                    0 <= nx < self.board_width
                    and 0 <= ny < self.board_height
                    and board[nx][ny] == 0
                ):
                    return True
        return False

    def get_score(self, komi: float = 5.5) -> dict:
        """
        Returns the area-scoring result as a dictionary, e.g.:
            {
              "white": { "stones": ..., "territory": ..., "komi": ..., "sum": ... },
              "black": { "stones": ..., "territory": ..., "komi": 0, "sum": ... },
            }

        Rules:
          - 1 point per stone
          - 1 point per empty intersection that is adjacent only to one color
          - White gets komi
        """
        # Count living stones (1 = black, 2 = white)
        black_stones = int((self.board == 1).sum())
        white_stones = int((self.board == 2).sum())

        # Find territory
        white_territory, black_territory = self._get_territory_counts()

        return {
            "white": {
                "stones": white_stones,
                "territory": white_territory,
                "komi": komi,
                "sum": white_stones + white_territory + komi,
            },
            "black": {
                "stones": black_stones,
                "territory": black_territory,
                "komi": 0,
                "sum": black_stones + black_territory,
            },
        }

    def _get_territory_counts(self) -> tuple[int, int]:
        """
        Traverse all empty intersections. For each connected region of empty points:
          - BFS/DFS over cells == 0
          - Gather which colors are adjacent to that region
          - If exactly one color is adjacent, that color owns all those empty intersections.
        """
        visited = set()
        white_territory = 0
        black_territory = 0

        for r in range(self.board_height):
            for c in range(self.board_width):
                if self.board[r, c] == 0 and (r, c) not in visited:
                    # BFS this region of empty cells
                    color_owner, region = self._flood_fill_territory(r, c, visited)
                    region_size = len(region)
                    if color_owner == 1:
                        black_territory += region_size
                    elif color_owner == 2:
                        white_territory += region_size
                    # else it's neutral => no one gets points

        return white_territory, black_territory

    def _flood_fill_territory(
        self, start_r: int, start_c: int, visited: set
    ) -> tuple[int | None, set[tuple[int, int]]]:
        """
        BFS starting from one empty cell. We'll track which colors (1=black, 2=white)
        appear around this region of empty cells. '#' squares (3) act like walls/edges:
        we do not cross them, but they do not by themselves add a color.

        Returns: (color_owner, region_set_of_cells)
          color_owner = 1 if only black is adjacent
                        2 if only white is adjacent
                        None if both or neither

        Example adjacency logic:
          - If we see black stones and no white stones, this region is black territory.
          - If we see white stones and no black stones, this region is white territory.
          - If we see both or none, we return None (neutral).
        """
        queue = deque()
        queue.append((start_r, start_c))
        region = set()
        adjacent_colors = set()

        while queue:
            r, c = queue.popleft()
            if (r, c) in visited:
                continue
            visited.add((r, c))

            # We only BFS through empty cells (val == 0).
            # If it's not empty (including #), we don't cross it.
            # But if it is a stone, we record its color in adjacent_colors.
            if self.board[r, c] == 0:
                region.add((r, c))
                # Check orthogonal neighbors
                for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < self.board_height and 0 <= nc < self.board_width:
                        neighbor_val = self.board[nr, nc]
                        if neighbor_val == 0:
                            # It's empty => keep BFS
                            if (nr, nc) not in visited:
                                queue.append((nr, nc))
                        elif neighbor_val in (1, 2):
                            # It's a stone => record color
                            adjacent_colors.add(neighbor_val)
                        # If neighbor_val == 3 (#), do nothing, skip it
            # If this cell isn't 0, we won't add more neighbors from it.

        if len(adjacent_colors) == 1:
            # Precisely one color in adjacency
            return adjacent_colors.pop(), region
        # If 0 or 2+ distinct colors => neutral
        return None, region

    def get_valid_moves(self, is_black: bool) -> np.ndarray:
        """
        Returns a 2D boolean array (same shape as the board)
        where each cell indicates if that (x,y) is a valid move for the given color (True/False).

        In your original JS: x => column index, y => row index.
        In your Python code: self.board is indexed [row, col].
        So be consistent. We'll treat:
          x = col, y = row
        or reverse. Just keep consistent with how you decode/encode actions.
        """
        color = 1 if is_black else 2
        valid_moves = np.zeros((self.board_height, self.board_width), dtype=bool)

        for r in range(self.board_height):
            for c in range(self.board_width):
                verdict = self.evaluate_if_move_is_valid(r, c, color)
                valid_moves[r, c] = verdict == GoValidity.VALID
        return valid_moves

    def is_game_over(self) -> bool:
        """
        For demonstration, lets say the game is over if self.last_player is None.
        Adjust to your real logic (e.g., two consecutive passes, etc.).
        """
        return (self.last_player is None) and (len(self.history) > 0)

    def evaluate_if_move_is_valid(
        self, row: int, col: int, player_color: int, shortcut: bool = True
    ) -> GoValidity:
        """
        Checks if placing a stone of 'player_color' at (row, col) is valid, approximating
        the logic from the JS code. Returns a GoValidity.

        Key checks:
         - If game over => return gameOver
         - If same color as last_player => notYourTurn
         - If out of bounds or blocked => pointBroken
         - If not empty => pointNotEmpty
         - Check for immediate easy pass with 'shortcut'
         - Otherwise try a full move simulation and see if it repeats the board or results in suicide.
        """
        # 1) Is the game over?
        if self.is_game_over():
            return GoValidity.GAME_OVER

        # 2) Turn check (last player can't play again, if you want strict alternation)
        if self.last_player == player_color:
            return GoValidity.NOT_YOUR_TURN

        # 3) Bounds / blocked / empty check
        if not (0 <= row < self.board_height and 0 <= col < self.board_width):
            return GoValidity.POINT_BROKEN

        if self.board[row, col] != 0:  # not empty
            return GoValidity.POINT_NOT_EMPTY

        # 4) The "possible repeat" check from JS. We'll only do a thorough check
        #    after we place a stone. But let's see if there's a direct clue:
        #    In the JS code, 'possibleRepeat' is a quick guess from the last boards.
        #    We'll just store the entire set of previous boards in `self.history`.
        #    We'll do the actual check after simulating the move (like the JS final step).
        #    So we won't do an early return for 'possibleRepeat' here.

        # Quick adjacency checks if shortcut == True
        if shortcut:
            # a) If the current point has any adjacent empty space => hasLiberty => valid (unless repeated)
            if self._has_direct_liberty(row, col):
                # We still need to confirm it's not repeated. So we do a partial approach:
                # We'll skip an early 'return GoValidity.valid' and do a final board check below.
                pass
            else:
                # b) If a friendly chain around (row, col) has more than 1 liberty => not suicide
                #    or if we can capture an enemy group => also not suicide.
                pass

        # 5) Let's do the full move simulation to check for suicide, captures, or repeated board states
        temp_board = self.board.copy()
        temp_board[row, col] = player_color
        # Attempt to capture any adjacent enemy groups that have no liberties
        opponent_color = 1 if player_color == 2 else 2
        self._capture_if_liberties_none(temp_board, row, col, opponent_color)

        # After potential captures, see if the newly placed stone is still on the board
        # and not part of a chain with zero liberties => if it's gone, it's suicide
        if temp_board[row, col] != player_color:
            # The newly placed stone got removed => suicide
            return GoValidity.NO_SUICIDE

        # 6) Check repeated board states
        new_board_str = self.decode_board(temp_board)
        if new_board_str in self.history:
            return GoValidity.BOARD_REPEATED

        # If we reach here, move is OK
        return GoValidity.VALID

    def _has_direct_liberty(self, row: int, col: int) -> bool:
        """Check if the empty cell (row,col) has any adjacent empty cell."""
        # For a single placed stone, a direct liberty means: any neighbor is empty => we can place safely.
        for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            rr, cc = row + dr, col + dc
            if 0 <= rr < self.board_height and 0 <= cc < self.board_width:
                if self.board[rr, cc] == 0:
                    return True
        return False

    def _capture_if_liberties_none(
        self, temp_board: np.ndarray, placed_r: int, placed_c: int, opponent_color: int
    ) -> None:
        """
        After placing a stone at (placed_r, placed_c), check each adjacent cell for opponent stones.
        If any opponent chain is now out of liberties, remove (capture) it.
        """
        visited = set()
        for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            rr, cc = placed_r + dr, placed_c + dc
            if 0 <= rr < self.board_height and 0 <= cc < self.board_width:
                if temp_board[rr, cc] == opponent_color and (rr, cc) not in visited:
                    # Find entire group
                    group = self._find_group(
                        temp_board, rr, cc, opponent_color, visited
                    )
                    # Check liberties
                    if not self._has_group_liberty(temp_board, group):
                        # remove them
                        for gx, gy in group:
                            temp_board[gx, gy] = 0

    def _find_group(
        self, board_arr: np.ndarray, sr: int, sc: int, color: int, visited: set
    ) -> set[tuple[int, int]]:
        """DFS or BFS to find all connected stones of 'color' from (sr, sc)."""
        stack = [(sr, sc)]
        group = set()
        while stack:
            r, c = stack.pop()
            if (r, c) in visited:
                continue
            if board_arr[r, c] != color:
                continue
            visited.add((r, c))
            group.add((r, c))
            for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                rr, cc = r + dr, c + dc
                if 0 <= rr < self.board_height and 0 <= cc < self.board_width:
                    if board_arr[rr, cc] == color and (rr, cc) not in visited:
                        stack.append((rr, cc))
        return group

    def _has_group_liberty(
        self, board_arr: np.ndarray, group: set[tuple[int, int]]
    ) -> bool:
        """Return True if any stone in the group has an adjacent empty cell."""
        for r, c in group:
            for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                rr, cc = r + dr, c + dc
                if 0 <= rr < board_arr.shape[0] and 0 <= cc < board_arr.shape[1]:
                    if board_arr[rr, cc] == 0:
                        return True
        return False

    # ---------------------------------------------------------------------
    # Example: How you might actually apply a move
    # ---------------------------------------------------------------------
    def play_move(self, row: int, col: int, player_color: int) -> bool:
        """
        Applies a move for player_color at (row,col), if valid.
        Returns True if the move was made, False if invalid.
        If made, updates self.board, self.last_player, and self.history.
        """
        verdict = self.evaluate_if_move_is_valid(row, col, player_color)
        if verdict != GoValidity.VALID:
            return False

        # Make a copy to finalize
        new_board = self.copy_board(self.board)
        new_board[row, col] = player_color
        # Attempt captures
        opponent_color = 1 if player_color == 2 else 2
        visited = set()
        for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            rr, cc = row + dr, col + dc
            if 0 <= rr < self.board_height and 0 <= cc < self.board_width:
                if new_board[rr, cc] == opponent_color and (rr, cc) not in visited:
                    group = self._find_group(new_board, rr, cc, opponent_color, visited)
                    if not self._has_group_liberty(new_board, group):
                        for gx, gy in group:
                            new_board[gx, gy] = 0

        # Update internal state
        self.board = new_board
        self.last_player = player_color
        self.history.insert(0, self.decode_board(self.board))
        return True

In [16]:
decoded_board = [".XX..", "O.OXX", ".O.O#", ".OOX.", "O.OX."]  # 10 17.5
decoded_board = ["..#O.", ".XOOO", "XOO.X", ".X..X", "...X."]  # 10, 12.5
decoded_board = ["..##O", "#XXX.", ".XO.O", ".XXX.", "O..OO"]  # 9 11.5
decoded_board = [".X.OO", "O..XX", ".OX.#", ".OOX.", "O.OX."]  # 9 16.5
decoded_board = ["..X..", ".....", "#OO.X", ".XO..", "X...."]  # 5, 8.5
go = Go(5, 5, decoded_board)

print(go.get_score())
# print(go)
# board_after, libs = go.state_after_action(go.encode_action(0, 2), False)
# print(rotate_and_beatify(board_after, "\n"))

{'white': {'stones': 3, 'territory': 0, 'komi': 5.5, 'sum': 8.5}, 'black': {'stones': 4, 'territory': 1, 'komi': 0, 'sum': 5}}
